In [25]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
le = LabelEncoder()

In [22]:
def label_encode_dataset(dataset):
    for column in dataset.columns.values:
        dataset[column] = dataset[column].astype(str)
        le.fit(y=dataset[column])
        dataset[column] = le.transform(dataset[column])
    return dataset

In [46]:
#import the data I need
test_data = pd.read_csv('csv_files/test.csv')
animal_data = pd.read_csv('csv_files/train.csv')

#Drop columns that I don't need
animal_data = animal_data.drop(labels=['AnimalID', 'Name', 'DateTime', 'OutcomeSubtype'], axis=1)

#setting up the test data
test_index = test_data.ID
test_data = test_data.drop(labels=['ID', 'Name', 'DateTime'], axis=1)
test_data = label_encode_dataset(test_data)

#splitting up the training data so I can use it to test how well my predictions are coming along
train_animal_data, test_animal_data = train_test_split(animal_data, test_size= .25, random_state=53)

#since the evaluation for this kaggle competition will be in multi-class logloss I've popped the outcomes off of the
#dataset and changed the results into dummy variables
outcome_train_animal_data = train_animal_data.pop('OutcomeType')
le.fit(outcome_train_animal_data)
outcome_train_animal_data = le.transform(outcome_train_animal_data)

#repeat for the test data
outcome_test_animal_data = test_animal_data.pop('OutcomeType')
le.fit(outcome_test_animal_data)
outcome_test_animal_data = le.transform(outcome_test_animal_data)

train_animal_data = label_encode_dataset(train_animal_data)
test_animal_data = label_encode_dataset(test_animal_data)

/Users/MatthewBarnette/shelter_animals/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/MatthewBarnette/shelter_animals/.direnv/python-3.5.0/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [63]:
dtrain = xgb.DMatrix(train_animal_data, label=outcome_train_animal_data)
dtest = xgb.DMatrix(test_animal_data, label=outcome_test_animal_data)

In [64]:
parm = {'bst:max_depth':2, 'bst:eta':1, 'silent':1, 'objective':'multi:softprob', 'num_class':5}
parm['nthread'] = 4
parm['eval_metric'] = 'auc'

In [65]:
evallist = [(dtest,'eval'), (dtrain,'train')]

In [66]:
num_round = 10
bst = xgb.train(parm,
                dtrain,
                num_round,
                evallist)

XGBoostError: b'AUC: the dataset only contains pos or neg samples'